In [3]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import randint, uniform

# --- Load ranked routes ---
ranked_routes = pd.read_csv('/Users/fahmi.taib/Desktop/Deployment Code Test/top_200_routes_by_ridership.csv')

# --- Load ridership data ---
ridership_df = pd.read_csv('/Users/fahmi.taib/Desktop/Deployment Code Test/very_new_finalised_ridership_route.csv', low_memory=False)
ridership_df['date'] = pd.to_datetime(ridership_df['date'], format='%m/%d/%y', errors='coerce')
ridership_df = ridership_df.dropna(subset=['date'])

# Append year to StartDate and EndDate, then parse as datetime
holiday_df['StartDate'] = holiday_df['StartDate'].astype(str) + '-2024'
holiday_df['EndDate'] = holiday_df['EndDate'].astype(str) + '-2024'

holiday_df['StartDate'] = pd.to_datetime(holiday_df['StartDate'], format='%d-%b-%Y', errors='coerce')
holiday_df['EndDate'] = pd.to_datetime(holiday_df['EndDate'], format='%d-%b-%Y', errors='coerce')

holiday_df['is_holiday'] = 1

# Create a list of all holiday dates between StartDate and EndDate
all_holiday_dates = []
for _, row in holiday_df.iterrows():
    if pd.isna(row['StartDate']) or pd.isna(row['EndDate']):
        continue
    all_holiday_dates.extend(pd.date_range(start=row['StartDate'], end=row['EndDate']))

# Convert to set for faster lookup
holiday_dates_set = set(all_holiday_dates)

# Then later in your ridership_df:
ridership_df['is_holiday'] = ridership_df['date'].apply(lambda x: 1 if x in holiday_dates_set else 0)


# Merge holiday info into ridership data
ridership_df['is_holiday'] = ridership_df['date'].apply(lambda x: 1 if x in holiday_dates_set else 0)


# --- Filter to top 180 routes ---
top_routes = ranked_routes.head(180)['route_no'].astype(str)
ridership_df = ridership_df[ridership_df['route_no'].astype(str).isin(top_routes)]

# --- Aggregate ridership per route per day ---
agg_df = ridership_df.groupby(['route_no', 'date']).agg({
    'ridership_total': 'sum',
    'hour': 'max',  # last hour of operation on that day for route
    'depot': 'first',
    'route_id': 'first',
    'is_holiday': 'first'
}).reset_index()

# --- Calculate hours left before schedule ends ---
END_HOUR = 24  # Adjust if your schedule ends at a different hour
agg_df['hours_left'] = END_HOUR - agg_df['hour']
agg_df['hours_left'] = agg_df['hours_left'].clip(lower=0)

# --- Add temporal features ---
agg_df['day_of_week'] = agg_df['date'].dt.dayofweek
agg_df['month'] = agg_df['date'].dt.month

# --- Encode categorical features ---
for col in ['depot', 'route_id', 'route_no']:
    agg_df[col + '_enc'], _ = pd.factorize(agg_df[col])

# --- Prepare features and target ---
features = [
    'route_no_enc', 'day_of_week', 'month',
    'depot_enc', 'route_id_enc', 'is_holiday', 'hours_left'
]
X = agg_df[features]
y = agg_df['ridership_total']

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- Define XGBoost model ---
xgb = XGBRegressor(random_state=42, objective='reg:squarederror')

# --- Hyperparameter distributions ---
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1)
}

# --- Randomized Search for hyperparameter tuning ---
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_mean_absolute_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

print("Starting hyperparameter tuning...")
random_search.fit(X_train, y_train)

print(f"Best parameters found: {random_search.best_params_}")

# --- Predict and evaluate ---
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nXGBoost Model Evaluation (Per Route Per Day with Hours Left):")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R-squared: {r2:.4f}")


Starting hyperparameter tuning...
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best parameters found: {'colsample_bytree': np.float64(0.7541666010159664), 'gamma': np.float64(0.07983126110107097), 'learning_rate': np.float64(0.0792681476866447), 'max_depth': 6, 'n_estimators': 466, 'reg_alpha': np.float64(0.6832635188254582), 'reg_lambda': np.float64(0.6099966577826209), 'subsample': np.float64(0.9332779646944658)}

XGBoost Model Evaluation (Per Route Per Day with Hours Left):
MAE: 96.26
MSE: 46678.16
RMSE: 216.05
R-squared: 0.9816


In [6]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint, uniform
import joblib

# --- Load ranked routes ---
ranked_routes = pd.read_csv('/Users/fahmi.taib/Desktop/Deployment Code Test/top_200_routes_by_ridership.csv')

# --- Load ridership data ---
ridership_df = pd.read_csv('/Users/fahmi.taib/Desktop/Deployment Code Test/very_new_finalised_ridership_route.csv', low_memory=False)
ridership_df['date'] = pd.to_datetime(ridership_df['date'], format='%m/%d/%y', errors='coerce')
ridership_df = ridership_df.dropna(subset=['date'])

# --- Load holiday data ---
holiday_df = pd.read_csv('/Users/fahmi.taib/Desktop/Deployment Code Test/Holiday 2024.csv')

# Append year and parse StartDate and EndDate
holiday_df['StartDate'] = holiday_df['StartDate'].astype(str) + '-2024'
holiday_df['EndDate'] = holiday_df['EndDate'].astype(str) + '-2024'

holiday_df['StartDate'] = pd.to_datetime(holiday_df['StartDate'], format='%d-%b-%Y', errors='coerce')
holiday_df['EndDate'] = pd.to_datetime(holiday_df['EndDate'], format='%d-%b-%Y', errors='coerce')

holiday_df['is_holiday'] = 1

# Create a set of all holiday dates
all_holiday_dates = []
for _, row in holiday_df.iterrows():
    if pd.isna(row['StartDate']) or pd.isna(row['EndDate']):
        continue
    all_holiday_dates.extend(pd.date_range(start=row['StartDate'], end=row['EndDate']))

holiday_dates_set = set(all_holiday_dates)

# Flag holidays in ridership data
ridership_df['is_holiday'] = ridership_df['date'].apply(lambda x: 1 if x in holiday_dates_set else 0)

# --- Filter to top 180 routes ---
top_routes = ranked_routes.head(180)['route_no'].astype(str)
ridership_df = ridership_df[ridership_df['route_no'].astype(str).isin(top_routes)]

# --- Aggregate ridership per route per day ---
agg_df = ridership_df.groupby(['route_no', 'date']).agg({
    'ridership_total': 'sum',
    'hour': 'max',  # last hour of operation on that day for route
    'depot': 'first',
    'route_id': 'first',
    'is_holiday': 'first'
}).reset_index()

# --- Calculate hours left before schedule ends ---
END_HOUR = 24  # Adjust if your schedule ends at a different hour
agg_df['hours_left'] = END_HOUR - agg_df['hour']
agg_df['hours_left'] = agg_df['hours_left'].clip(lower=0)

# --- Add temporal features ---
agg_df['day_of_week'] = agg_df['date'].dt.dayofweek
agg_df['month'] = agg_df['date'].dt.month

# --- Encode categorical features ---
for col in ['depot', 'route_id', 'route_no']:
    agg_df[col + '_enc'], _ = pd.factorize(agg_df[col])

# --- Prepare features and target ---
features = [
    'route_no_enc', 'day_of_week', 'month',
    'depot_enc', 'route_id_enc', 'is_holiday', 'hours_left'
]
X = agg_df[features]
y = agg_df['ridership_total']

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Define XGBoost model ---
xgb = XGBRegressor(random_state=42, objective='reg:squarederror')

# --- Hyperparameter distributions ---
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1)
}

# --- Randomized Search for hyperparameter tuning ---
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_mean_absolute_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

print("Starting hyperparameter tuning...")
random_search.fit(X_train_scaled, y_train)

print(f"Best parameters found: {random_search.best_params_}")

# --- Predict and evaluate ---
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nXGBoost Model Evaluation (Per Route Per Day with Hours Left):")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R-squared: {r2:.4f}")

# --- Save model and scaler ---
import joblib

joblib.dump(best_model, '/Users/fahmi.taib/Desktop/Deployment Code Test/xgb_ridership_model.pkl')
joblib.dump(scaler, '/Users/fahmi.taib/Desktop/Deployment Code Test/xgb_feature_scaler.pkl')

print("Model and scaler saved successfully.")


Starting hyperparameter tuning...
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best parameters found: {'colsample_bytree': np.float64(0.9660854910505922), 'gamma': np.float64(2.211761148655522), 'learning_rate': np.float64(0.08193620774722096), 'max_depth': 8, 'n_estimators': 147, 'reg_alpha': np.float64(0.18286599710730733), 'reg_lambda': np.float64(0.9346139973397097), 'subsample': np.float64(0.8553082375373402)}

XGBoost Model Evaluation (Per Route Per Day with Hours Left):
MAE: 91.84
MSE: 36646.74
RMSE: 191.43
R-squared: 0.9856
Model and scaler saved successfully.


In [16]:
import pandas as pd

ridership_path = "/Users/fahmi.taib/Desktop/Deployment Code Test/very_new_finalised_ridership_route.csv"
depot_mapping_path = "/Users/fahmi.taib/Desktop/Deployment Code Test/mapping_depot.csv"

ridership_df = pd.read_csv(ridership_path)
depot_df = pd.read_csv(depot_mapping_path)

print("Rows in ridership_df before mapping:", len(ridership_df))
print("Rows in depot_df:", len(depot_df))

# Create a mapping series from depot_df: keys=line_id, values=depot
# If there are duplicates on line_id in depot_df, it picks the first occurrence
line_id_to_depot = depot_df.drop_duplicates(subset=['line_id']).set_index('line_id')['depot']

# Add new column by mapping line_id from ridership_df to depot_id_new from depot_df
ridership_df['depot_id_new'] = ridership_df['line_id'].map(line_id_to_depot)

print("Rows in ridership_df after mapping:", len(ridership_df))

output_path = "/Users/fahmi.taib/Desktop/Deployment Code Test/v2_very_new_finalised_ridership_with_depot.csv"
ridership_df.to_csv(output_path, index=False)

print(f"New column 'depot_id_new' added and saved to {output_path}")


Rows in ridership_df before mapping: 1044737
Rows in depot_df: 17503
Rows in ridership_df after mapping: 1044737
New column 'depot_id_new' added and saved to /Users/fahmi.taib/Desktop/Deployment Code Test/v2_very_new_finalised_ridership_with_depot.csv


In [15]:
print("Rows in ridership_df before merge:", len(ridership_df))
print("Rows in depot_df:", len(merged_df))

Rows in ridership_df before merge: 1044737
Rows in depot_df: 48126222
